<a href="https://colab.research.google.com/github/b2200765028/AIN311-Project-P01/blob/main/ain311_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tqdm import tqdm
from datasets import list_datasets, load_dataset
from pprint import pprint
import random
import keras
import os

In [ ]:
dataset = load_dataset('squad') ## downloading squad dataset

Found cached dataset squad (C:/Users/Huawei/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453)


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
dataset = dataset.shuffle(seed=42)
dataset

Loading cached shuffled indices for dataset at C:/Users/Huawei/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453\cache-10de9997c4b83f65.arrow
Loading cached shuffled indices for dataset at C:/Users/Huawei/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453\cache-f3a033b6ac26514f.arrow


DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

In [ ]:
train_squad = pd.DataFrame(dataset["train"][:2000])
test_squad = pd.DataFrame(dataset["validation"][:2000])

In [ ]:
train_squad.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        2000 non-null   object
 1   title     2000 non-null   object
 2   context   2000 non-null   object
 3   question  2000 non-null   object
 4   answers   2000 non-null   object
dtypes: object(5)
memory usage: 78.2+ KB


In [ ]:
train_squad.head()
## we dont need id gonna drop it

,id,title,context,question,answers
0,573173d8497a881900248f0c,Egypt,The Pew Forum on Religion & Public Life ranks ...,What percentage of Egyptians polled support de...,"{'text': ['84%'], 'answer_start': [468]}"
1,57277e815951b619008f8b52,"Ann_Arbor,_Michigan",The Ann Arbor Hands-On Museum is located in a ...,Ann Arbor ranks 1st among what goods sold?,"{'text': ['books'], 'answer_start': [402]}"
2,5727e2483acd2414000deef0,Rule_of_law,One important aspect of the rule-of-law initia...,"In developing countries, who makes most of the...","{'text': ['the executive'], 'answer_start': [6..."
3,5728f5716aef0514001548cc,Samurai,"In December 1547, Francis was in Malacca (Mala...",Who impressed Xavier by taking notes in church?,"{'text': ['Anjiro'], 'answer_start': [160]}"
4,572826002ca10214002d9f16,Group_(mathematics),Groups are also applied in many other mathemat...,What represents elements of the fundamental gr...,"{'text': ['loops'], 'answer_start': [489]}"


In [ ]:
train_squad["answers"][12]["text"]

['1970s']

In [ ]:
train_squad.drop("id",axis=1,inplace=True)
test_squad.drop("id",axis=1,inplace=True)

In [ ]:
train_squad["title"].value_counts()

2008_Sichuan_earthquake                         24
New_York_City                                   18
Beyoncé                                         17
Queen_Victoria                                  17
Roman_Republic                                  16
                                                ..
National_Archives_and_Records_Administration     1
Pesticide                                        1
General_Electric                                 1
Turner_Classic_Movies                            1
Glacier                                          1
Name: title, Length: 416, dtype: int64

In [ ]:
avg_words_context = round(sum([len(i.split()) for i in train_squad["context"]])/len(train_squad))
avg_words_question = round(sum([len(i.split()) for i in train_squad["question"]])/len(train_squad))
avg_words_context,avg_words_question

(120, 10)

In [ ]:
max_words_context = (max([len(i.split()) for i in train_squad["context"]]))
max_words_question = (max([len(i.split()) for i in train_squad["question"]]))
max_words_context,max_words_question

(486, 26)

## Tokenization

In [ ]:
encoder_text = train_squad["context"]
decoder_text = train_squad["question"]

In [ ]:
from keras.preprocessing.text import Tokenizer


tokenizer = Tokenizer()
text = encoder_text + decoder_text
tokenizer.fit_on_texts(text)
dictionary = tokenizer.word_index
    
word2i ={k:v for (k,v) in dictionary.items()}
i2word = {v:k for (k,v) in word2i.items()}

In [ ]:
print("Most occured 10 words in vocab")
sorted(tokenizer.word_counts,key=tokenizer.word_counts.get,reverse=True)[:10]

Most occured 10 words in vocab


['the', 'of', 'and', 'in', 'to', 'a', 'is', 'was', 'as', 'for']

In [ ]:
vocab_size= len(word2i)+1
vocab_size

26185

In [ ]:
encoder_seq = tokenizer.texts_to_sequences(encoder_text)
decoder_seq = tokenizer.texts_to_sequences(decoder_text)

In [ ]:
print("Original Text \n-----------\n",encoder_text[10])
print("Turned into sequences \n-----------\n",encoder_seq[10])

Original Text 
-----------
 In July 1855, the allied squadron tried to go past Taganrog to Rostov on Don, entering the Don River through the Mius River. On 12 July 1855 HMS Jasper grounded near Taganrog thanks to a fisherman who moved buoys into shallow water. The Cossacks captured the gunboat with all of its guns and blew it up. The third siege attempt was made 19–31 August 1855, but the city was already fortified and the squadron could not approach close enough for landing operations. The allied fleet left the Gulf of Taganrog on the 2nd September 1855, with minor military operations along the Azov Sea coast continuing until late autumn 1855.
Turned into sequences 
-----------
 [4, 340, 3458, 1, 1378, 6289, 1665, 5, 811, 1074, 7584, 5, 9734, 14, 5398, 5399, 1, 5398, 308, 88, 1, 14021, 308, 14, 434, 340, 3458, 9735, 9736, 4715, 456, 7584, 4220, 5, 6, 14022, 34, 673, 14023, 51, 4221, 270, 1, 14024, 1666, 1, 14025, 13, 52, 2, 32, 2382, 3, 6290, 21, 114, 1, 290, 3459, 934, 8, 99, 14026, 

### we are going to find max length of sequences and I will use padding to make them equal size

In [ ]:
max_len_enc = 0
max_len_dec = 0

for enc,dec in zip(encoder_seq,decoder_seq):
    if len(enc) > max_len_enc:
        max_len_enc = len(enc)
    if len(dec) > max_len_dec:
        max_len_dec= len(dec)

In [ ]:
max_len_enc,max_len_dec

(498, 26)

In [ ]:
from keras.utils import pad_sequences
encoder_data = pad_sequences(encoder_seq,maxlen=max_len_enc,dtype="int32",padding="post",truncating="post")
decoder_data = pad_sequences(decoder_seq,maxlen=max_len_dec,dtype="int32",padding="post",truncating="post")

#### as we can see it added extra zeros to make them same size(we selected post to add zeros after)

In [ ]:
encoder_data[0][40:200]

array([   32,  3160,   929,     2,   182,    12,  1047,   810,  3788,
         196,     5,     1,   687,     3,  1238,     2,  6264,     2,
         277,  1370,  4202,     4,    22, 13960,    11,     1,    84,
         181,     5,     6,   301,  7560,   518,  4695,  1969,  6265,
           2,  4696,  9702,   707,     1,   302,  1656,    10,   155,
          34,  2539,  1657,  4203,   707, 13961,     3,  4204,   465,
           2,  2929,    10,  7561,     3, 13962,     3,  6266,   210,
       13963,     6,   617,    34,  9703,  9704,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,

### Pretrained embedding using glove

In [ ]:
path_to_glove_file = os.path.join(
 "./glove.6B.100d.txt"
)

embeddings_index = {}
with open(path_to_glove_file,encoding="UTF-8") as f:
    for line in f:
        word, weights = line.split(maxsplit=1)
        weights = np.fromstring(weights,sep=" ",)
        embeddings_index[word] = weights
print(line)
print("Found %s word vectors." % len(embeddings_index))

sandberger 0.28365 -0.6263 -0.44351 0.2177 -0.087421 -0.17062 0.29266 -0.024899 0.26414 -0.17023 0.25817 0.097484 -0.33103 -0.43859 0.0095799 0.095624 -0.17777 0.38886 0.27151 0.14742 -0.43973 -0.26588 -0.024271 0.27186 -0.36761 -0.24827 -0.20815 0.22128 -0.044409 0.021373 0.24594 0.26143 0.29303 0.13281 0.082232 -0.12869 0.1622 -0.22567 -0.060348 0.28703 0.11381 0.34839 0.3419 0.36996 -0.13592 0.0062694 0.080317 0.0036251 0.43093 0.01882 0.31008 0.16722 0.074112 -0.37745 0.47363 0.41284 0.24471 0.075965 -0.51725 -0.49481 0.526 -0.074645 0.41434 -0.1956 -0.16544 -0.045649 -0.40153 -0.13136 -0.4672 0.18825 0.2612 0.16854 0.22615 0.62992 -0.1288 0.055841 0.01928 0.024572 0.46875 0.2582 -0.31672 0.048591 0.3277 -0.50141 0.30855 0.11997 -0.25768 -0.039867 -0.059672 0.5525 0.13885 -0.22862 0.071792 -0.43208 0.5398 -0.085806 0.032651 0.43678 -0.82607 -0.15701

Found 400000 word vectors.


In [ ]:
num_tokens = vocab_size + 2 ## not sure why it is +2
embedding_dim = 100  ## 
hits = 0
misses = 0

embedding_matrix = np.zeros((num_tokens, embedding_dim))

In [ ]:
for word, i in word2i.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1

In [ ]:
len(embedding_matrix)

26187

In [ ]:
len(word2i)

26184

In [ ]:
print("Converted %d words (%d misses)" % (hits, misses))

In [ ]:
from tensorflow.keras.layers import Embedding

embedding_layer = Embedding(
    num_tokens,
    embedding_dim,
    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=False,
)

In [ ]:
num_samples = len(encoder_seq)
num_samples

In [ ]:
## encoder_data = encoder_data.reshape(-1,2000,498)
## decoder_data = decoder_data.reshape(-1,2000,26)

In [ ]:
encoder_data =  tf.convert_to_tensor(encoder_data,dtype=float)
decoder_data = tf.convert_to_tensor(decoder_data,dtype=float)
encoder_data

In [ ]:

'''from keras.models import Model
from keras.layers import Input, LSTM, Dense

latent_dim = 128
encoder_inputs = Input(shape=(2000,498))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_data)

encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(2000,26))

# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the 
# return states in the training model, but we will use them in inference.

decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_data,
                                     initial_state=encoder_states)
decoder_dense = Dense(max_len_dec, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
'''

In [ ]:
latent_dim = 128

encoder_inputs = Input(shape=(2000,498))
print(encoder_data.shape)
x = embedding_layer(encoder_inputs)
print(x.shape)
x, state_h, state_c = LSTM(latent_dim,
                           return_state=True)(x)
print(x.shape)
encoder_states = [state_h, state_c]



decoder_inputs = Input(shape=(2000,26))
x = embedding_layer(decoder_data)
print(x.shape)
x = LSTM(latent_dim, return_sequences=True)(x, initial_state=encoder_states)
print(x.shape)
decoder_outputs = Dense(128, activation='softmax')(x)


model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [ ]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
# Note that `decoder_target_data` needs to be one-hot encoded,
# rather than sequences of integers like `decoder_input_data`!
model.fit([encoder_data, decoder_data], decoder_outputs,
          epochs=5)
What if I don't want to use teacher forcing for training?

In [ ]:
tf.convert_to_tensor(encoder_inputs)